In [23]:
import pandas as pd
import glob
import warnings
import os
import requests
import gspread
from dotenv import load_dotenv

load_dotenv()
base_id = "appEX0yBBP1xhAkeL"
api_key = "keyedN52o4Usdo8DS"
table_name = 'USERS'
AIRTABLE_BASE_ID=os.environ.get(base_id)
AIRTABLE_API_KEY=os.environ.get(api_key)
AIRTABLE_TABLE_NAME=os.environ.get(table_name)
AIRTABLE_URL = f"https://api.airtable.com/v0{AIRTABLE_BASE_ID}"
warnings.filterwarnings('ignore')

# Archivo Google Sheets
sa = gspread.service_account(filename = './gspread/service_account.json')
sh = sa.open('Users TUDUU')
wks = sh.worksheet('automatizacion')

json = wks.get_all_records()
df = pd.DataFrame(json)

load_dotenv()

path = r'cluster_2_1/' # use your path
#save into df all files
all_files = glob.glob(path + "/*.xlsx")

In [24]:
# Para conectarnos a AirTable

#!pip install pyairtable
#!pip airtable install airtable-python-wrapper
from pyairtable import Table
table = Table(api_key, base_id, table_name)

In [26]:
tipos_consumidores = pd.DataFrame(pd.DataFrame(table.all(fields = [' Email', 'Tipo de consumidor']))['fields'].tolist())

In [27]:
tipos_consumidores

,Email,Tipo de consumidor
0,golgolgol@gmail.com,NaN
1,itejadavazquez@gmail.com,NaN
2,sara.martinezz@alumni.mondragon.edu,NaN
3,paula.sevillano@alumni.mondragon.edu,NaN
4,a@o.com,NaN
...,...,...
80,alberto.camarero@alumni.mondragon.edu,NaN
81,glfkddd@gmail.com,NaN
82,teja@gmail.com,NaN
83,aiora.iturmendi@gmail.com,NaN


In [112]:
# los clusters los deciden los propios clientes a la hora de hacer el test
import numpy as np
tipos_consumidores['cluster'] = ''
tipos_consumidores['cluster']  = np.where(tipos_consumidores['Tipo de consumidor']=='Como muy sano y hago deporte', str(1), tipos_consumidores['cluster'])
tipos_consumidores['cluster']  = np.where(tipos_consumidores['Tipo de consumidor']=='Dieta equilibrada, sin excesos', str(2), tipos_consumidores['cluster'])
tipos_consumidores['cluster']  = np.where(tipos_consumidores['Tipo de consumidor']=='Como sano pero me gusta caer en la tentación', str(3), tipos_consumidores['cluster'])
tipos_consumidores['cluster']  = np.where(tipos_consumidores['Tipo de consumidor']=='No me quito nada, como de todo', str(4), tipos_consumidores['cluster'])
# Ahora hay muchos nan porque al registrarnos no hemos hecho el test, con los registros reales no 
# debería de haber ningún nan; todos tienen que hacer el test
# Los de prueba los metemos en el cluster 4
tipos_consumidores['cluster'] = tipos_consumidores['cluster'].fillna(str(4))

0      
1      
2      
3      
4      
     ..
83     
84     
85     
86     
87     
Name: cluster, Length: 88, dtype: object

In [ ]:
# hasta aquí hemos llegado a conseguir la información del tipo de consumidor; ahora, tendríamos que conseguir comprobar qué usuario es el que ha entrado nuevo, es decir, en qué usuario detecta el "change" y coger el tipo de consumidor que le pertenece a ese usuario.

In [1]:
import gspread
import time
import pandas as pd
import os
import requests
from dotenv import load_dotenv

sa = gspread.service_account(filename = './gspread/service_account.json')
sh = sa.open('Users TUDUU')
wks = sh.worksheet('automatizacion')
wks2 = sh.worksheet('productos')
json = wks.get_all_records()
json2 = wks2.get_all_records()
df = pd.DataFrame(json)
df_prod = pd.DataFrame(json2)

load_dotenv()
AIRTABLE_BASE_ID=os.environ.get("appEX0yBBP1xhAkeL")
AIRTABLE_API_KEY=os.environ.get("keyedN52o4Usdo8DS")
AIRTABLE_TABLE_NAME=os.environ.get("Cabeceras")
AIRTABLE_TABLE_NAME2=os.environ.get("Users")

In [16]:
df2 = pd.DataFrame(wks.get_all_records())

In [29]:
df

,user,email
0,cesta,user
1,cesta,user
2,cesta,user
3,cesta,user
4,cesta,user
5,cesta,user
6,cesta,user
7,cesta,user
8,cesta,user
9,cesta,user


In [28]:
df2[~df2.isin(df)].dropna()['Email']

KeyError: ' Email'

In [14]:
while True:
    df2 = pd.DataFrame(wks.get_all_records())
    if df.equals(df2):
        print('No change')
        
    else:
        print('Change')
        # print changes
        print(df2[~df2.isin(df)].dropna())
        #get values
        changes = df2[~df2.isin(df)].dropna()
        print(changes)
        df = df2

        # get change

        # run local script

        endpoint=f'https://api.airtable.com/v0/appEX0yBBP1xhAkeL/Cabeceras'


        headers = {
            "Authorization": f"Bearer keyedN52o4Usdo8DS ",
            "Content-Type": "application/json"
        }

        data = {
        "records": [
            {
            "fields": {
                "Cesta": changes['cesta'].values[0],
                "Email": changes['Email'].values[0]
            }
            }
        ]
        }
        r = requests.post(endpoint, json=data, headers=headers)
        print('llega')
        
        record_id = r.json()['records'][0]['id']
        #save record id to use in the next script
        print(record_id)
        print(r.status_code)
        os.system('python final_model.py')
        

        
    time.sleep(5)

No change


KeyboardInterrupt: 